## Защита данных клиентов страховой компании "Хоть потоп"

### Описание проекта

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Также необходимо обосновать корректность его работы.

Требуется защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

### План проекта

1. Загрузить и изучить данные.


2. Ответить на вопрос и обосновать решение:
    - Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? 
        (Её можно обучить заново)
        - Изменится. Привести примеры матриц.
        - Не изменится. Указать, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
        

3. Предложить алгоритм преобразования данных для решения задачи. Обосновать, почему качество линейной регрессии не поменяется.


4. Запрограммировать этот алгоритм, применив матричные операции. Проверить, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

### Описание данных

    Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
    
    Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## 1. Загрузка данных

Импортируем необходимые библиотеки и массив данных. Посмотрим на общую информацию о нем.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from numpy.linalg import inv
from sklearn.utils import shuffle
from IPython.display import display
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [1]:
df = pd.read_csv('/datasets/insurance.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


5000 строк, 5 столбцов. Пропусков не видно. Изучаем дальше.

In [2]:
df.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


Столбцы "возраст" и "зарплата" можно привети к целочисленному значению.

In [3]:
df['Возраст'] = df['Возраст'].astype('int')
df['Зарплата'] = df['Зарплата'].astype('int')

Проверим на необычные значения.

In [4]:
df['Пол'].value_counts()

0    2505
1    2495
Name: Пол, dtype: int64

Полов, как это неудивительно, два, и этот признак даже почти сбалансирован.

In [5]:
df['Возраст'].sort_values()

2688    18
3370    18
1159    18
2549    18
1693    18
        ..
3117    60
2240    60
3907    61
4019    62
228     65
Name: Возраст, Length: 5000, dtype: int64

Диапазон возраста от 18 до 65 - все правдиво.

In [6]:
df['Зарплата'].sort_values()

726      5300
4164     6000
4623     7400
437      8900
483      9800
        ...  
2193    71400
3328    71600
4360    74800
4512    75200
3255    79000
Name: Зарплата, Length: 5000, dtype: int64

Зарплата в диапазоне от, увы, 5300 до 79000 - нет поводов для сомнений.

In [7]:
df['Члены семьи'].sort_values()

2432    0
1132    0
3769    0
3772    0
1129    0
       ..
1944    6
4809    6
3320    6
384     6
4710    6
Name: Члены семьи, Length: 5000, dtype: int64

От нуля до 6 домочадцев - также вполне реалистично.

In [8]:
df['Страховые выплаты'].sort_values()

0       0
3231    0
3229    0
3228    0
3227    0
       ..
3907    4
3209    4
3674    4
3117    4
228     5
Name: Страховые выплаты, Length: 5000, dtype: int64

От нуля до пяти страховых выплат за последние пять лет. Клиент с индексом 228 либо очень невезуч, либо очень прагматичен.

Проверим на наличие дубликатов.

In [9]:
df.duplicated().sum()

153

153 дубликата. То есть гипотетически 153 клиента, у которых совпали значения всех пяти признаков. Достаточно маловероятно. Необходимо от них избавиться.

In [10]:
df.drop_duplicates().reset_index(drop = True)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0
...,...,...,...,...,...
4842,0,28,35700,2,0
4843,0,34,52400,1,0
4844,0,20,33900,2,0
4845,1,22,32700,3,0


Дубликаты удалены.

Вывод: данные подготовлены, приступаем к работе над следующей частью.

## 2. Умножение матриц

Нам необходимо ответить на вопрос: признаки умножают на обратимую матрицу - изменится ли качество линейной регрессии? 

Для ответа на этот вопрос необходимо привести математическое доказательство.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии не изменится, так как предсказания до умножения равны предсказаниям после умножения.

**Обоснование:** 

Нам необходимо доказать, что $$a = a'$$ Следовательно $$Xw = X'w'$$

$$
Xw = X(X^T X)^{-1} X^T y
$$

$$
X'w' = XP((XP)^T XP)^{-1} (XP)^T y
$$

$$
X(X^T X)^{-1} X^T y = XP((XP)^T XP)^{-1} (XP)^T y
$$

Делим обе части уравнения на X и y

$$
(X^T X)^{-1} X^T  = P((XP)^T XP)^{-1} (XP)^T 
$$

Сокрщаем -1 степень

$$
X^T/(X^TX) = P(XP)^T/((XP)^TXP)
$$

Сокращаем прочие переменные.

$$
1/X = 1/X
$$

$$
X = X
$$

Что и требовалось доказать.


## 3. Алгоритм преобразования

**Алгоритм**

Оптимальным способом для решения поставленной задачи видится умножение имеющейся матрицы на что-либо случайно полученное. 

Наша матрица состоит из 4847 строк и 5 колонок. 

Умножать на вектор не имеет смысла, так как результатом умножения будет новый вектор, из которого будет невозможно восстановить данные. 

Умножать на на случайно полученное число тоже не имеет смысла, так как теоритически его можно будет подобрать перебором.

Поэтому логичным будет умножать на случайную обратимую матрицу высотой в 5 строк, так как исходный массив шириной в 5 столбцов. 

**Обоснование**

В этом случае вероятность подбора матрицы-ключа для  получения скрываемой информации будет стремиться к нулю, а мы же со своей стороны можем продолжать изучение линейной регрессии без потери качества на уже измененной матрице.

## 4. Проверка алгоритма

Получим случайную обратимую матрицу.

In [11]:
matrix = np.random.normal(0, 100, size = (4, 4))
matrix

array([[ -3.5668187 , 152.15108797, 115.76398824, -20.49908117],
       [-10.41195488, -19.83246471, -74.54895113, 133.40406901],
       [147.05820874, 129.69464587,  20.77098466, -20.17126045],
       [-82.17770856,  61.29447079, 104.18979539, -12.32425999]])

Проверим ее на обратимость.

In [12]:
matrix_r = inv(matrix)
matrix_r

array([[-0.04082228,  0.00186839,  0.02882419,  0.04094767],
       [ 0.05186741, -0.00119777, -0.02835113, -0.05283424],
       [-0.06657677,  0.00327783,  0.04195416,  0.07755196],
       [-0.03267974,  0.0092955 ,  0.02147972,  0.03867895]])

Ошибки нет, так что матрица обратима. Перейдем к преобразованию исходных данных. Преобразовывать будем только признаки.

In [13]:
df_trans = df.drop('Страховые выплаты', axis=1)
df_trans = df_trans.dot(matrix)
df_trans.head()

,0,1,2,3
0,7.293575e+06,6.432255e+06,1.027404e+06,-995057.774664
1,5.587651e+06,4.927546e+06,7.859724e+05,-760383.634057
2,3.087920e+06,2.723012e+06,4.340288e+05,-419727.751378
3,6.131944e+06,5.407973e+06,8.647929e+05,-838364.723695
4,3.837924e+06,3.384627e+06,5.401511e+05,-522755.082806


Данные преобразованы. Теперь попробуем вернуть исходный массив. Для этого нам необходимо полученный массив умножить на обратимую матрицу matrix_r.

In [14]:
checking = df_trans.dot(matrix_r)
checking.head()

,0,1,2,3
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,-5.917760e-11,46.0,38000.0,1.000000e+00
2,-5.046933e-11,29.0,21000.0,2.081627e-12
3,-9.909594e-12,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,-1.701058e-11


Сравним полученные данные с исходными. Сразу выделим отдельно признаки исходного датафрейма для этого сравнения и убодства в дальнейшем.

In [15]:
df_check = pd.DataFrame(checking)
df_check = df_check.round()
df_check = df_check.astype('int64')
df_check.columns = ['Пол', 'Возраст', 'Зарплата', 'Члены семьи']
features = df.drop('Страховые выплаты', axis=1)
print('Исходный массив')
display(features.head())
print('Массив после преобразования')
display(df_check.head())
print('Сравнение')
display(features.head() == df_check.head())

Исходный массив


,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


Массив после преобразования


,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


Сравнение


,Пол,Возраст,Зарплата,Члены семьи
0,True,True,True,True
1,True,True,True,True
2,True,True,True,True
3,True,True,True,True
4,True,True,True,True


Сравнение проведено, массивы совпадают.

Теперь требуется установить на практике, что качество линейной регрессии не изменится после преобразования. Используем метрику R2.

В первую очередь разделим признаки и целевой признак. Целевой признак, согласно условию, будем использовать один и тот же.

In [16]:
target = df['Страховые выплаты']
features_t = df_trans

Теперь получим две модели линейной регрессии, обучим их и сравним метрику R2.

In [17]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.42494550308169177


In [18]:
model_t = LinearRegression()
model_t.fit(features_t, target)
predictions_t = model_t.predict(features_t)
print(r2_score(target, predictions_t))

0.4249455030817054


R2 совпадает, преобразование успешно, гипотеза доказана, цели достигнуты.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования